In [111]:
#Preamble stuff, loading up libraries and convenience functions
import all_data_handler
import pandas
import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols
import itertools
import seaborn as sb
import scipy.stats as ss #For ss.f_oneway() ANOVAs
import re

adh = all_data_handler.UserData()
per_task_df = pandas.DataFrame(adh.toPandas())
col_list = list(per_task_df)
col_list.remove("user")
col_list.remove("condition")
col_list.remove("task")

#Given the object list of a gesture from the coding, return a list of letters representing normalized 
#values for the objects of the gesture
def tag_object(original):
    #This use of strip is to prevent quotes from messing up regex matches
    original = " ".join(original).strip("\"")
    tags = []
    #Matches robots, robot, bot, bots, etc. 
    robots = re.compile("bot|group|swarm|orange|red", re.I)
    crate = re.compile("crate", re.I)
    targetA = re.compile("area a|box a| a$|to a,|^a$", re.I)
    targetB = re.compile("area b|box b| b$|to b,|^b$", re.I)
    whitespace = re.compile("whitespace|ground|screen|white area", re.I)
    
    toCheck = [(robots, "r"), (crate, "c"), (targetA, "a"), (targetB, "b"), (whitespace, "w")]
    for compiled, tag in toCheck:
        if re.search(compiled, original):
            tags.append(tag)
    return tags

def count_select_taps(participant):
    #Dict of task id to count of taps on robots
    counts = {}
    for task_id in participant["tasks"].keys():
        #Default to no counts of select taps
        counts[task_id] = 0
        events = participant["tasks"][task_id]
        #Flip through all the events
        for event in events:
            #If we have objects, tag them
            if "objects" in event.keys():
                tags = tag_object(event["objects"])
                #We have both an event and a set of tags, check if it's a tap on a robot
                if 'r' in tags and event["event_type"] == "tap":
                    counts[task_id] += 1
    return counts

def count_group_select(participant):
    #Dict of task id to count of taps on robots
    counts = {}
    for task_id in participant["tasks"].keys():
        #Default to no counts of select taps
        counts[task_id] = 0
        events = participant["tasks"][task_id]
        #Flip through all the events
        for event in events:
            #If we have objects, tag them
            if "objects" in event.keys():
                tags = tag_object(event["objects"])
                #We have both an event and a set of tags, check if it's a tap on a robot
                if 'r' in tags and (event["event_type"] == "lasso" or event["event_type"] == "box_select"):
                    counts[task_id] += 1
    return counts

def count_box_select(participant):
    #Dict of task id to count of taps on robots
    counts = {}
    for task_id in participant["tasks"].keys():
        #Default to no counts of select taps
        counts[task_id] = 0
        events = participant["tasks"][task_id]
        #Flip through all the events
        for event in events:
            #If we have objects, tag them
            if "objects" in event.keys():
                tags = tag_object(event["objects"])
                #We have both an event and a set of tags, check if it's a tap on a robot
                if 'r' in tags and event["event_type"] == "box_select":
                    counts[task_id] += 1
    return counts

def count_lasso_select(participant):
    #Dict of task id to count of taps on robots
    counts = {}
    for task_id in participant["tasks"].keys():
        #Default to no counts of select taps
        counts[task_id] = 0
        events = participant["tasks"][task_id]
        #Flip through all the events
        for event in events:
            #If we have objects, tag them
            if "objects" in event.keys():
                tags = tag_object(event["objects"])
                #We have both an event and a set of tags, check if it's a tap on a robot
                if 'r' in tags and event["event_type"] == "lasso":
                    counts[task_id] += 1
    return counts

#Returns a dict of task IDs to count of total events
def count_events(participant):
    counts = {}
    for task_id in participant["tasks"].keys():
        #Default to no counts of select taps
        counts[task_id] = 0
        #Flip through all the events
        events = participant["tasks"][task_id]
        for event in events:
            if event["event_type"] == "memo":
                #This event is a memo, not a user gesture
                continue
            elif "example" in event.keys() and event["example"] == True:
                #This event is an example, don't count it
                continue
            else:
                #Not one of the skip cases, so count it
                counts[task_id] += 1
    return counts

#Prints out the per-condition counts, one entry for each user in that condition
#Also gets the mean and standard deviation for the condition
def user_counts(user_counts):
    per_cond = {}
    #For each user, maintain a running total
    for pid in user_counts.keys():
        #Get their condition
        cond = adh.IdToCondition(pid)[0]
        counts = user_counts[pid].values()
        per_user_total = sum(counts)
        if cond in per_cond.keys():
            per_cond[cond].append(per_user_total)
        else:
            per_cond[cond] = [per_user_total]
    
    for cond in per_cond.keys():
        
        total = sum(per_cond[cond])
        mean = np.mean(per_cond[cond])
        std_dev = np.std(per_cond[cond])
        print cond, per_cond[cond]
        print "total:{} mean:{} std dev:{}".format(total, mean, std_dev)
        print

#Given a dict of user IDs to dicts of task ids to event counts
#return a dict of user IDs to lists of event counts for the common tasks
def get_matched_tasks(counts):
    #Get a list of the task names that every condition has in common
    common_names = []
    for name in adh.taskMap.keys():
        if all(adh.taskMap[name].values()):
            common_names.append(name)
    common_names.sort()

    #dict of user ids to a list of counts, index of counts is task number
    matched_tasks = {}
    for user in counts.keys():
        common_counts = []
        for task in common_names:
            #print user, adh.IdToCondition(user), task, adh.taskNameToNumber(task, user)
            #Get the count at the task number for this common name
            common_counts.append(counts[user][str(adh.taskNameToNumber(task, user))])
        matched_tasks[user] = common_counts
    return matched_tasks

#Given a dict of conditions to lists of samples
#Do an all-pairs 1-way ANOVA on the samples
def all_pairs_f(d):
    for k1, k2 in itertools.combinations(d.keys(), 2):
        print k1, k2
        print ss.f_oneway(d[k1], d[k2])
        print

#Given a dict of users to lists of samples
#Return a dict of conditions to a list of all the samples for users in that condition 
def make_condition_dict(user_samples):
    cond_dict = {}
    for user in user_samples.keys():
        #Get the condition for this user
        cond = adh.IdToCondition(user)[0]
        #If we already have a set of samples, extend it, otherwise, create it
        if cond in cond_dict.keys():
            cond_dict[cond].extend(user_samples[user])
        else:
            cond_dict[cond] = user_samples[user]
    return cond_dict

#Set up dicts of user ids to dicts of task ids to various types of counts
all_select_taps = adh.apply(count_select_taps)
all_group_selects = adh.apply(count_group_select)
all_lassos = adh.apply(count_lasso_select)
all_box = adh.apply(count_box_select)
all_gesture_counts = adh.apply(count_events)

H1: There exists a number of robots beyond which users will transition from treating robots as individuals to interacting with the robots in small groups or as a single large group. 

This transition point will be apparent because of a change in the gesture set that the user uses to interact with the swarm. It is hypothesized that above the transition point, users will be more likely to neglect some subset of the available robots. The user will instead use commands that control the bulk of the robots as a cloud or flock, but may leave some robots unused. For example, the user may switch from selecting robots as individuals to shaping and pushing the swarm the way a child might play with a bug, putting their hand down so the bug goes around or avoids it, touching the back of the bug gently to make it scurry forwards, and so forth, or by shaping the group as if sculpting, with pushing and pinching to "carry" groups around. 

The user may also change how they indicate which robots are to be interacted with. Rather than selecting each robot by clicking on it, they may "paint" over the area containing the robots they want to use, or draw a circle around them. 
The size of the swarm where changes in the user gestures occur will indicate the transition point between interacting with individual robots and interacting with the swarm as a whole. 

This hypothesis would be invalidated by the gestures selected by the user displaying no correlation with the size of the swarm that they are controlling.

In [112]:
#Filter user gesture counts, user tap-as-select counts, and user group select counts to common tasks only
common_select_tap = get_matched_tasks(all_select_taps)
common_group_select = get_matched_tasks(all_group_selects)
common_gesture_counts = get_matched_tasks(all_gesture_counts)

#For each user, get the sum of their gesture use across all the common tasks. This is the user's "verbosity".
#The verbosity is used to normalize the raw counts of gestures so that users that make a lot of gestures don't
#dominate the analysis. 
total_gesture_counts = {k:sum(v) for (k,v) in common_gesture_counts.items()}

#Normalize tap-as-select for matched tasks
#This is a list comprehension that does the norming (divide each task by the appropriate entry in the total counts)
#inside of a dictionary comprehension that operates over all users
norm_select_tap = {k:[float(count)/float(total_gesture_counts[k]) for count in v] for (k,v) in common_select_tap.items()}
#Normalize group select for matched tasks
norm_group_select = {k:[float(count)/float(total_gesture_counts[k]) for count in v] for (k,v) in common_group_select.items()}

#Set up dictionaries by condition for all-pairs ANOVA for tap-as-select and group select
select_tap_by_cond = make_condition_dict(norm_select_tap)
group_select_by_cond = make_condition_dict(norm_group_select)

#And do our ANOVAS
print "Tap as Select"
all_pairs_f(select_tap_by_cond)



Tap as Select
unknown thousand
F_onewayResult(statistic=10.018005437298077, pvalue=0.0018236967394952741)

unknown hundred
F_onewayResult(statistic=14.705522441068645, pvalue=0.00017424816114220467)

unknown ten
F_onewayResult(statistic=0.7287942882840502, pvalue=0.3944216675361002)

unknown one
F_onewayResult(statistic=2.9023566766190023, pvalue=0.09019530837884651)

thousand hundred
F_onewayResult(statistic=1.4082174869703987, pvalue=0.2369339307835794)

thousand ten
F_onewayResult(statistic=4.90458227749459, pvalue=0.0280549461959977)

thousand one
F_onewayResult(statistic=14.185654486179072, pvalue=0.0002249002445095893)

hundred ten
F_onewayResult(statistic=8.397717434413893, pvalue=0.004229147672432684)

hundred one
F_onewayResult(statistic=17.044437317400693, pvalue=5.604839366083625e-05)

ten one
F_onewayResult(statistic=5.339063046821974, pvalue=0.021999336496061207)



For a=0.05, the following pairs of conditions in bold have statistically significant differences in the use of tap as select:

|              | unknown | one    | ten       | hundred     | thousand   | 
| ------------ | ------- | ------ | --------- | ----------- | ---------- |
| **unknown**  |         | 0.0902 | 0.3944    | **0.0002**  | **0.0018** |   
| **one**      |         |        | **0.022** | **0.0000**  | **0.0002** |
| **ten**      |         |        |           | **0.0042**  | **0.0280** |
| **hundred**  |         |        |           |             | 0.2369     |
| **thousand** |         |        |           |             |            |

One and ten not differing from unknown is expected. In the one and ten robot case, actions on individual robots do not take up too much time, and the unknown case is like the one robot case in that a single entity is displayed, although it represents more than one robot. 

Hundred and thousand, on the other hand, do not differ much from each other because they are both cases in which there are too many robots to individually select each one by tapping on it. 

In [113]:
print "Group Select"
all_pairs_f(group_select_by_cond)

Group Select
unknown thousand
F_onewayResult(statistic=10.817674837981038, pvalue=0.001211720122771659)

unknown hundred
F_onewayResult(statistic=49.25952085126323, pvalue=4.536563960870581e-11)

unknown ten
F_onewayResult(statistic=35.277081680347536, pvalue=1.4681319808005658e-08)

unknown one
F_onewayResult(statistic=nan, pvalue=nan)

thousand hundred
F_onewayResult(statistic=30.711663423934183, pvalue=1.0640175529130789e-07)

thousand ten
F_onewayResult(statistic=20.48419900796339, pvalue=1.0968165989616746e-05)

thousand one
F_onewayResult(statistic=10.817674837981038, pvalue=0.001211720122771659)

hundred ten
F_onewayResult(statistic=0.5695052357987681, pvalue=0.451452861890829)

hundred one
F_onewayResult(statistic=49.25952085126323, pvalue=4.536563960870581e-11)

ten one
F_onewayResult(statistic=35.27708168034752, pvalue=1.4681319808005658e-08)



For a=0.05, the following pairs of conditions in bold have statistically significant differences in the use of group selections:

|              | unknown | one    | ten        | hundred     | thousand   | 
| ------------ | ------- | ------ | ---------- | ----------- | ---------- |
| **unknown**  |         | NaN    | **0.0000** | **0.0000**  | **0.0012** |   
| **one**      |         |        | **0.0000** | **0.0000**  | **0.0012** |
| **ten**      |         |        |            |   0.4514    | **0.0000** |
| **hundred**  |         |        |            |             | **0.0000** |
| **thousand** |         |        |            |             |            |

The reason that unknown and one have an uncomputable ANOVA is that in the common tasks for the one and unknown robot conditions, no group selection gestures were used.

This restriction to the common tasks is because some tasks do not make sense for a single robot, such as forming a square formation, and some do not make sense for an unknown number of robots, such as tasks that single out a specific robot. 

In the next section, I will work with only the ten, hundred, and thousand robot cases, as these cases have all of their tasks in common (but not in common with the one and unknown cases). 



In [114]:
#Convert the dictionary of user id to dictionary of task to count to a simple list of counts
#The list is ordered by task
def tasks_to_list(tasks):
    counts = []
    for key in sorted(tasks.iterkeys()):
        counts.append(tasks[key])
    return counts

#For each user, get the sum of their gesture use across all the tasks. As above, this gets used as verbosity.
total_gesture_counts = {k:sum([count for count in v.values()]) for (k,v) in all_gesture_counts.items()}

all_select_taps
#Convert the dicts of per-task dicts of counts into a dict of ordered list of counts
#The keys at the top level remain the user ids
tap_select_counts = {k:tasks_to_list(v) for (k,v) in all_select_taps.items()}
group_select_counts = {k:tasks_to_list(v) for (k, v) in all_group_selects.items()}

# #Normalize the counts by the user's total gestures
norm_select_tap = {k:[float(count)/float(total_gesture_counts[k]) for count in v] for (k,v) in tap_select_counts.items()}
norm_group_select = {k:[float(count)/float(total_gesture_counts[k]) for count in v] for (k,v) in group_select_counts.items()}

#Set up dictionaries by condition for all-pairs ANOVA for tap-as-select and group select
select_tap_by_cond = make_condition_dict(norm_select_tap)
group_select_by_cond = make_condition_dict(norm_group_select)

#Get rid of the conditions we're not checking (the don't have the same number of samples)
del select_tap_by_cond["one"]
del select_tap_by_cond["unknown"]
all_pairs_f(select_tap_by_cond)



thousand hundred
F_onewayResult(statistic=0.5735700464985992, pvalue=0.4493419280174722)

thousand ten
F_onewayResult(statistic=7.657093144630474, pvalue=0.005948624582697074)

hundred ten
F_onewayResult(statistic=4.611193497471902, pvalue=0.032434907331346775)



Across all tasks, with a=0.05, hundred and thousand both differ from ten, but not from each other, in the use of taps as a selection gesture. This is likely because with ten robots, tapping each individual is not too difficult, but beyond that, it becomes time-consuming and tedious. 

In [115]:
#Get rid of the conditions we're not checking (the don't have the same number of samples)
del group_select_by_cond["one"]
del group_select_by_cond["unknown"]
all_pairs_f(group_select_by_cond)


thousand hundred
F_onewayResult(statistic=40.698644495921904, pvalue=5.493434764274091e-10)

thousand ten
F_onewayResult(statistic=27.38660707018453, pvalue=2.843748424876117e-07)

hundred ten
F_onewayResult(statistic=1.5621537031224022, pvalue=0.21216659061140616)



Across all tasks, with a=0.05, hundred and ten both differ from thousand, but not from each other, in the use of group selects as a selection gesture. 

The reason for this is that the use of group selections actually drops off in the thousand robot case compared to the hundred robot case. This is evidence that the user expects commands to be obeyed by all robots, without having to select them first. 


In [116]:
def total_by_condition(by_user):
    counts_by_cond = make_condition_dict(by_user)
    for cond in counts_by_cond.keys():
        print cond, sum(counts_by_cond[cond])

#Display the total count of group selection gestures per condition
print "\n-- group select, all tasks --"
total_by_condition(group_select_counts)

#Display the total amount of group selection gestures within the common tasks, per condition
print "\n-- group select, common tasks --"
total_by_condition(common_group_select)

#Display the total amount of tap selection gestures per condition
print "\n-- tap select, all tasks --"
total_by_condition(tap_select_counts)

#Display the total amount of tap selection gestures within the common tasks, per condition
print "\n-- tap select, common tasks --"
total_by_condition(common_select_tap)


-- group select, all tasks --
unknown 1
thousand 34
hundred 121
ten 131
one 3

-- group select, common tasks --
unknown 0
thousand 15
hundred 69
ten 69
one 0

-- tap select, all tasks --
unknown 43
thousand 22
hundred 35
ten 140
one 78

-- tap select, common tasks --
unknown 25
thousand 8
hundred 4
ten 64
one 59


Note that as described above, the thousand robot task has a lower use of group selections than the ten and hundred robot cases, in both the common and in all tasks. It also has a lower use of tap select than the ten and hundred robot cases over all all tasks, but twice the use of tap to select in the common tasks. 

This diminished use of group select for thousand robots across all tasks is what accounts for the statistically significant difference between ten and thousand and between hundred and thousand cases for group selection, while having a statistically insignificant difference between hundred and ten. 


H2: A display which obscures individual robots and displays a cloud or swarm boundary will cause the user to treat the swarm as a whole rather than individuals, which will be apparent because the user will use the same gestures they would use to control a single robot. 

Once the ratio of the size of individual swarm members to the size of the area the swarm is in becomes sufficiently large, displaying the swarm members at the same scale as the map will result in the representation of the swarm members being too small to interact with. This problem will arise at smaller scales if the swarm robots are themselves quite tiny, and some of the available swarm robots are indeed small. Scaling the representation of the robots up, relative to the map, will make the robot representations overlap unrealistically and obscure the map. Instead, we propose that for certain scales of swarms, it makes sense to represent the swarm as the area covered, rather than the locations of the individual robots. This approach has been used successfully for navigation in three dimensions, by developing a controller that causes the individual UAVs to remain within a bounding prism, and allowing the user to control the shape and location of that prism.

Altering how the user interface displays the location of the robots in the swarm will affect the transition point. 

This hypothesis would be invalidated by the gestures selected by the user in the single robot case being dissimilar from those selected in the case where the swarm is displayed as a cloud or covered region. However, some variation is expected in commands for tasks which a swarm can do, which a single robot cannot perform, such as dividing into two groups. 


In [117]:
#Modified version of get_matched_tasks to get the tasks that are in common 
#specifically between the one robot case and the unknown number of robots case
#I suspect these are the same as the common names, but just to be sure...
def get_one_unknown_tasks(counts):
    #Get a list of the task names that every condition has in common
    common_names = []
    for name in adh.taskMap.keys():
        if all([adh.taskMap[name]['X'], adh.taskMap[name][1]]):
            common_names.append(name)
    common_names.sort()

    #dict of user ids to a list of counts, index of counts is task number
    matched_tasks = {}
    for user in counts.keys():
        common_counts = []
        for task in common_names:
            #print user, adh.IdToCondition(user), task, adh.taskNameToNumber(task, user)
            #Get the count at the task number for this common name
            common_counts.append(counts[user][str(adh.taskNameToNumber(task, user))])
        matched_tasks[user] = common_counts
    return matched_tasks

#Filter user gesture counts, user tap-as-select counts, and user group select counts to common tasks only
common_select_tap = get_one_unknown_tasks(all_select_taps)
common_group_select = get_one_unknown_tasks(all_group_selects)
common_gesture_counts = get_one_unknown_tasks(all_gesture_counts)

#For each user, get the sum of their gesture use across all the common tasks. This is the user's "verbosity".
#The verbosity is used to normalize the raw counts of gestures so that users that make a lot of gestures don't
#dominate the analysis. 
total_gesture_counts = {k:sum(v) for (k,v) in common_gesture_counts.items()}

#Normalize tap-as-select for matched tasks
#This is a list comprehension that does the norming (divide each task by the appropriate entry in the total counts)
#inside of a dictionary comprehension that operates over all users
norm_select_tap = {k:[float(count)/float(total_gesture_counts[k]) for count in v] for (k,v) in common_select_tap.items()}
#Normalize group select for matched tasks
norm_group_select = {k:[float(count)/float(total_gesture_counts[k]) for count in v] for (k,v) in common_group_select.items()}

#Set up dictionaries by condition for all-pairs ANOVA for tap-as-select and group select
select_tap_by_cond = make_condition_dict(norm_select_tap)
group_select_by_cond = make_condition_dict(norm_group_select)

#And do our ANOVAS
print "Tap as Select"
print ss.f_oneway(select_tap_by_cond["one"], select_tap_by_cond["unknown"])
print "Group Select"
print ss.f_oneway(group_select_by_cond["one"], group_select_by_cond["unknown"])


Tap as Select
F_onewayResult(statistic=2.902356676619004, pvalue=0.09019530837884651)
Group Select
F_onewayResult(statistic=nan, pvalue=nan)


This is exactly the same as above, when looking at the common tasks. The common tasks across all tasks are the same as the common tasks between the one and unknown conditions, because the one and unknown conditions are the limiting factors on the size of the set of common tasks.

These values are copied from the earlier all-pairs ANOVAs of the common tasks, they are exactly the same as the result from above. 

(for tap as select)
> unknown one
>
> F_onewayResult(statistic=2.9023566766190023, pvalue=0.09019530837884651)

(for group select)
> unknown one
>
> F_onewayResult(statistic=nan, pvalue=nan)

It's good that they are NaN because they're exactly the same for group selections, and not statistically significantly different for tap gestures, because it indicates that for all selection gestures, the unknown and one robot conditions are similar. 

What about for all gestures?

In [122]:
#Now we're working with pandas! 🐼

#Initialize the list of common names for tasks in both the unknown condition and the one robot condition
common_names = []
for name in adh.taskMap.keys():
    if all([adh.taskMap[name]['X'], adh.taskMap[name][1]]):
        common_names.append(name)
common_names.sort()

#Add a total gestures column to the data frame that has the total count of gestures
#Each row of the dataframe is a user doing a task, so the count will be the total count of gestures used in that task
per_task_df["total_gestures"] = per_task_df[col_list].sum(axis=1)

per_task_df

#Get all the one or unknown condition entries
one_cond = per_task_df.query("condition=='one'")
unk_cond = per_task_df.query("condition == 'unknown'")


#This gives me the sum of all of the gestures within users, across tasks
one_cond.groupby("user").sum()
#one_cond
    
    

,box,doubletap,drag,draw,hold,lasso,other,pinch,rev_pinch,tap,tripletap,ui,voice,total_gestures
user,,,,,,,,,,,,,,
12,0,0,10,2,0,2,0,0,0,0,0,0,0,14
17,0,0,14,0,1,0,0,0,0,1,0,0,0,16
2,0,1,10,1,1,0,0,0,1,14,0,11,0,39
22,0,3,10,0,0,0,0,0,0,3,0,4,4,24
27,0,0,12,3,0,1,0,0,0,2,0,0,0,18
32,0,27,0,17,4,0,1,0,0,53,0,4,0,106
37,0,0,1,13,2,0,0,0,0,12,0,0,0,28
42,0,2,17,6,2,0,0,0,2,15,3,0,2,49
47,0,2,14,0,4,0,0,0,2,4,1,0,0,27


In [124]:
#This gives me the sum of each gesture, within tasks, across users
one_cond.groupby("task").sum()


,box,doubletap,drag,draw,hold,lasso,other,pinch,rev_pinch,tap,tripletap,ui,voice,total_gestures
task,,,,,,,,,,,,,,
crate,0,3,15,1,0,0,0,0,0,10,0,2,2,33
divide_color_1,0,0,21,9,1,0,0,0,0,6,1,0,0,38
divide_color_2,0,0,20,2,1,0,0,0,0,4,0,0,0,27
mark,0,4,0,3,3,2,1,0,0,6,1,5,0,25
move_a,0,0,10,4,0,0,0,0,0,7,0,0,0,21
move_wall,0,0,8,2,0,0,0,0,0,0,0,0,0,10
patrol_a,0,7,10,5,2,0,0,0,1,31,0,4,1,61
patrol_screen,0,9,8,2,2,0,0,0,1,18,0,3,1,44
remove,0,4,4,6,1,1,0,0,0,4,0,4,0,24
